In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
from sagemaker.tensorflow import TensorFlow
import numpy as np
import sagemaker

# Local Execution Example - Record Mode

In [6]:
role='arn:aws:iam::708267171719:role/sagemaker-local'

hyperparameters={'steps':12000, # 600 steps per epoch, 20 epochs
                  'batch_size':100,
                  'learning_rate':0.001,
                  'verbosity':'INFO'} 

tf_estimator = TensorFlow(py_version='py3', 
                          framework_version='1.12', 
                          entry_point='task.py',
                          role=role,
                          train_instance_count=1,
                          train_instance_type='local',
                          hyperparameters=hyperparameters,
                          source_dir='training_job/')

In [7]:
tf_estimator.fit({'train':'s3://sagemaker-us-east-2-708267171719/sagemaker/ml-model-migration/data/mnist/train',
                  'eval':'s3://sagemaker-us-east-2-708267171719/sagemaker/ml-model-migration/data/mnist/test'})




Creating tmpo75qhg_4_algo-1-j1kee_1 ... 
Attaching to tmpo75qhg_4_algo-1-j1kee_12mdone
algo-1-j1kee_1  | 2019-07-16 17:33:52,984 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-j1kee_1  | 2019-07-16 17:33:53,021 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-j1kee_1  | 2019-07-16 17:33:53,575 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-j1kee_1  | 2019-07-16 17:33:53,605 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-j1kee_1  | 2019-07-16 17:33:53,622 sagemaker-containers INFO     Invoking user script
algo-1-j1kee_1  | 
algo-1-j1kee_1  | Training Env:
algo-1-j1kee_1  | 
algo-1-j1kee_1  | {
algo-1-j1kee_1  |     "additional_framework_parameters": {},
algo-1-j1kee_1  |     "channel_input_dirs": {
algo-1-j1kee_1  |         "train": "/opt/ml/input/data/train",
algo-1-j1kee_1  |         "eval": "/opt/ml/input/data/eval"
algo-1-j1kee

# Local Endpoint Deployment

In [8]:

mnist_predictor = tf_estimator.deploy(initial_instance_count=1,
                                      instance_type='local')

W0716 12:49:10.490144 4612814272 connectionpool.py:662] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13d32b048>: Failed to establish a new connection: [Errno 61] Connection refused')': /ping
W0716 12:49:10.496733 4612814272 connectionpool.py:662] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13d349438>: Failed to establish a new connection: [Errno 61] Connection refused')': /ping
W0716 12:49:10.498295 4612814272 connectionpool.py:662] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13d349860>: Failed to establish a new connection: [Errno 61] Connection refused')': /ping


Attaching to tmppiwg2o_m_algo-1-8kyni_1
algo-1-8kyni_1  | INFO:__main__:starting services
algo-1-8kyni_1  | INFO:__main__:using default model name: model
algo-1-8kyni_1  | INFO:__main__:tensorflow serving model config: 
algo-1-8kyni_1  | model_config_list: {
algo-1-8kyni_1  |   config: {
algo-1-8kyni_1  |     name: "model",
algo-1-8kyni_1  |     base_path: "/opt/ml/model",
algo-1-8kyni_1  |     model_platform: "tensorflow"
algo-1-8kyni_1  |   },
algo-1-8kyni_1  | }
algo-1-8kyni_1  | 
algo-1-8kyni_1  | 
algo-1-8kyni_1  | INFO:__main__:nginx config: 
algo-1-8kyni_1  | load_module modules/ngx_http_js_module.so;
algo-1-8kyni_1  | 
algo-1-8kyni_1  | worker_processes auto;
algo-1-8kyni_1  | daemon off;
algo-1-8kyni_1  | pid /tmp/nginx.pid;
algo-1-8kyni_1  | error_log  /dev/stderr info;
algo-1-8kyni_1  | 
algo-1-8kyni_1  | worker_rlimit_nofile 4096;
algo-1-8kyni_1  | 
algo-1-8kyni_1  | events {
algo-1-8kyni_1  |   worker_connections 2048;
algo-1-8kyni_1  | }
algo-1-8kyni_1  | 
algo-1-8kyni_1 

In [11]:
from keras.datasets import mnist
def load_mnist_data():   
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = np.reshape(x_train, [-1, 28,28,1]).astype(np.float32)
    x_test = np.reshape(x_test, [-1, 28,28,1]).astype(np.float32)
    x_train /= 255
    x_test /= 255
    train_data = {'images':x_train, 'labels':y_train}
    test_data = {'images':x_test, 'labels':y_test}
    return train_data, test_data

train_data, test_data = load_mnist_data()

for ex in range(1,10):
    # load an example from the test set
    example = test_data['images'][ex].reshape(1,28,28,1)
    #predictions is a dict{'predictions'[[]]}
    predictions = mnist_predictor.predict(example)
    #predictions['predictions'][0] contains the softmax activations of the network
    predicted_label = np.argmax(predictions['predictions'][0])
    label = test_data['labels'][ex]
    print("Example {}: Predicted label: {}  Actual label:{}".format(ex, predicted_label, label))

algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 152 "-" "-"
Example 1: Predicted label: 2  Actual label:2
Example 2: Predicted label: 1  Actual label:1
algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 151 "-" "-"
Example 3: Predicted label: 0  Actual label:0
algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 152 "-" "-"
Example 4: Predicted label: 4  Actual label:4
algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 155 "-" "-"
Example 5: Predicted label: 1  Actual label:1
algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 151 "-" "-"
Example 6: Predicted label: 4  Actual label:4
algo-1-8kyni_1  | 172.18.0.1 - - [16/Jul/2019:17:52:07 +0000] "POST /invocations HTTP/1.1" 200 157 "-" "-"
Example 7: Predicted label: 9  Actual label:9
algo-1-8kyni_1  | 172.18.0.1 - - [16